<a href="https://colab.research.google.com/github/JYP97/DS2_Proj_Jobs_skills_analysis/blob/master/BERT_BACKEND_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.7 MB/s eta 0:00:00


In [4]:
# Load tokenizer & fine-tuned model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
print("=== BERT tokenizer loaded. ===")
model = BertForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/DataScience/DS2/acc_models_0.51_0.49_warmup100_2e-5_666", 
    num_labels = 8
)
print("=== Fine-tuned model loaded. ===")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

=== BERT tokenizer loaded. ===
=== Fine-tuned model loaded. ===


In [24]:
# Let users input their skills and turn the input into tokens
input_skills = input("\n\nPlease input your skills: ")
encodings = tokenizer.encode_plus(
    input_skills,
    # None,
    add_special_tokens=True,
    max_length = 512,           # Pad & truncate all sentences.
    pad_to_max_length = True,
    # truncation=True,
    return_attention_mask = True,   # Construct attn. masks.
    return_tensors = 'pt',
)



Please input your skills: c++,python,java


In [25]:
labels = ['Managers',
 'Professionals',
 'Service and sales workers',
 'Plant and machine operators and assemblers',
 'Craft and related trades workers',
 'Technicians and associate professionals',
 'Clerical support workers',
 'Elementary occupations']

In [26]:
model.eval()
with torch.no_grad():
    input_ids=encodings.input_ids
    attention_mask=encodings.attention_mask
    token_type_ids=encodings.token_type_ids
    output=model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output.logits).cpu().detach().numpy().tolist()
    print(int(np.argmax(final_output, axis=1)))

1


In [8]:
final_output

[[0.22045005857944489,
  0.08654899895191193,
  0.38413581252098083,
  0.5524672269821167,
  0.37568792700767517,
  0.7083438634872437,
  0.7720301151275635,
  0.9022297263145447]]

In [27]:
from itertools import chain
probabilities=list(chain.from_iterable(final_output))
predictions = dict(zip(labels,probabilities))
print(max(predictions, key=predictions.get))

Professionals
